In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-yba9re5a/unsloth_35ca774ee7244fcbaef7c12da51cb7c6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-yba9re5a/unsloth_35ca774ee7244fcbaef7c12da51cb7c6
  Resolved https://github.com/unslothai/unsloth.git to commit a100afa60cf1566e784b41eb462b5d2ed80977f8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5kit6mm4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5kit6mm4
  Resolved https://github.com/huggingface/transformers.git to commit 76a33a10923ccc1074917f6b6a1e719e626b7dc9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.40.0.dev0-py3-none-any.whl size=8802691 sha256=7b819e6a2e4ed4e2cc3329d95775e2514f7d8be612c59406f736e9588b825dbe
  Stored in directory: /tmp/pip-ephem-wheel-cache-a5qktgyh/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [4]:
!pip install -U datasets -q

!pip install trl -q

In [5]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import torch

from unsloth import FastLanguageModel
from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from transformers import TrainingArguments

In [6]:

import random
import numpy as np
import transformers
from datasets import Dataset
from datasets import DatasetDict

from transformers import AutoTokenizer
from datasets import ClassLabel, Sequence
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split

from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

In [7]:
dataset = pd.read_csv('/content/drive/MyDrive/NER_CS/NER_Datas.csv')

In [8]:
dataset.head()

,tokens,tags
0,ORF,O
1,aspnet_client,O
2,casino,O
3,4.8,I-version
4,3.6.3,B-version


In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder to your tags column and transform it
dataset['label'] = label_encoder.fit_transform(dataset['tags'])

# Display the mapping of original tags to labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:")
print(label_mapping)



Label Mapping:
{'B-PL': 0, 'B-application': 1, 'B-cve id': 2, 'B-edition': 3, 'B-file': 4, 'B-function': 5, 'B-hardware': 6, 'B-language': 7, 'B-method': 8, 'B-os': 9, 'B-parameter': 10, 'B-relevant_term': 11, 'B-update': 12, 'B-vendor': 13, 'B-version': 14, 'I-application': 15, 'I-edition': 16, 'I-hardware': 17, 'I-os': 18, 'I-relevant_term': 19, 'I-update': 20, 'I-vendor': 21, 'I-version': 22, 'O': 23}


In [10]:
dataset.head()

,tokens,tags,label
0,ORF,O,23
1,aspnet_client,O,23
2,casino,O,23
3,4.8,I-version,22
4,3.6.3,B-version,14


In [11]:
dataset.drop(columns=['tags'], inplace=True)
dataset.head()

,tokens,label
0,ORF,23
1,aspnet_client,23
2,casino,23
3,4.8,22
4,3.6.3,14


In [12]:
 # Split the data into training, validation, and test sets
train, X_temp = train_test_split(dataset, test_size=0.3, random_state=42)
validation, test = train_test_split(X_temp, test_size=0.10, random_state=42)

In [13]:
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)
test = test.reset_index(drop=True)

In [14]:
 # Create a datasets.Dataset object
train = Dataset.from_dict(train)
validation = Dataset.from_dict(validation)
test = Dataset.from_dict(test)
# Print the dataset to check the format
print("Train Dataset : ",train)
print("Validation Dataset : ", validation)
print("Test Dataset : ", test)

Train Dataset :  Dataset({
    features: ['tokens', 'label'],
    num_rows: 18078
})
Validation Dataset :  Dataset({
    features: ['tokens', 'label'],
    num_rows: 6974
})
Test Dataset :  Dataset({
    features: ['tokens', 'label'],
    num_rows: 775
})


In [15]:
# Create DatasetDict
dataset = DatasetDict({
    'validation': validation,
    'test': test,
    'train': train
})
print(dataset)

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'label'],
        num_rows: 6974
    })
    test: Dataset({
        features: ['tokens', 'label'],
        num_rows: 775
    })
    train: Dataset({
        features: ['tokens', 'label'],
        num_rows: 18078
    })
})


In [16]:
dataset['test']['tokens'][774]

'Synology'

In [17]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # True to use 4bit quantization / reduce memory usage (for T4 GPU)

In [18]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistralai/Mistral-7B-v0.1", #"unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Unsloth: You passed in `mistralai/Mistral-7B-v0.1` and `load_in_4bit = True`.
We shall load `unsloth/mistral-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

### Add LoRa adapters

In [19]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # can improve fine-tuning, at attention/feed fwd layers
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16, # more change to pre-train weights but care overfitting
    lora_dropout = 0.05, # any, but 0 if perf opti.
    bias = "none",    # any, but "none" is perf  opti.
    use_gradient_checkpointing = True,
    random_state = 11,
    use_rslora = False,  # support rank stabilized LoRA
    loftq_config = None, # LoftQ support
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Data Preparation (Alpaca template)

In [20]:
prompt = """Develop a token classification model to accurately identify and classify named entities label:
### Token:
{}
### label:
{}"""

In [21]:
# Add EOS special token, according to previously loaded tokenizer
EOS_TOKEN = tokenizer.eos_token
def format_prompts(examples):
    inputs = examples["tokens"]
    outputs = examples["label"]
    texts = []
    for inp, output in zip(inputs, outputs):
        # Add end of string token to prevent infinite generations.
        text = prompt.format(inp, output) + EOS_TOKEN
        texts.append(text)
    return {"text":texts,}

In [22]:
# Build prompts
dataset['train'] = dataset['train'].map(format_prompts, batched = True,)

# Print a sample :
dataset['train'][3]['text']

Map:   0%|          | 0/18078 [00:00<?, ? examples/s]

'Develop a token classification model to accurately identify and classify named entities label:\n### Token:\n6.1.4\n### label:\n14</s>'

In [23]:
dataset['validation'] = dataset['validation'].map(format_prompts, batched = True,)

Map:   0%|          | 0/6974 [00:00<?, ? examples/s]

In [24]:
label_list = ['B-PL', 'B-application', 'B-cve id', 'B-edition', 'B-file',
              'B-function', 'B-hardware', 'B-language', 'B-method', 'B-os',
              'B-parameter', 'B-relevant_term', 'B-update', 'B-vendor',
              'B-version', 'I-application', 'I-edition', 'I-hardware',
              'I-os', 'I-relevant_term', 'I-update', 'I-vendor', 'I-version', 'O']

### Train the model

In [28]:
training_args = TrainingArguments(
    per_device_train_batch_size = 24,
    gradient_accumulation_steps = 24,
    warmup_steps = 5,
    num_train_epochs = 3,
    # max_steps = 110,
    learning_rate = 2e-4, # 2e-5
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 11,
    output_dir = "/content/drive/MyDrive/NER_CS/Mistral",
    run_name = "mistral-7b-base_en_v11",
    logging_strategy = 'steps',
    logging_steps = 93,
    evaluation_strategy='steps',
    save_strategy="steps",
    save_steps=93,
    save_total_limit = 2,
    )

In [29]:
# init trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['validation'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # if packing = False, else default to None
    packing = False, # more speed if packing short sequences. Maybe later
    args = training_args
)

Map (num_proc=2):   0%|          | 0/18078 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/6974 [00:00<?, ? examples/s]

In [30]:
# train
trainer = trainer.train()

Step,Training Loss,Validation Loss
93,0.818100,0.559165


### Inference

Run the model we just trained. With Unsloth `FastLanguageModel`, test on one sample.

In [31]:
print(dataset['test'][22])
comment = dataset['test'][22]['tokens']
print(comment)

{'tokens': 'superfluous', 'label': 23}
superfluous


#### One sample

In [32]:
# load LoRa adapters from HuggingFace
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/NER_CS/Mistral/checkpoint-93",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [43]:
print(f"{dataset['test'][0]}\n")
import re
inputs = tokenizer(
    [
        prompt.format(
        dataset['test']['tokens'][0], # insert comment
        "", # output - blank (instead of label) for generation
        )
    ],
    return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
decoded_outputs = tokenizer.batch_decode(outputs)
# decoded_outputs
decoded_outputs = str(decoded_outputs)
decoded_outputs
# # Using regular expression to find the label value starting with '###'
# label_match = re.search(r'### label:\n(\d+)', decoded_outputs)

# if label_match:
#     label_value = label_match.group(1)
#     print("Label value:", label_value)
# else:
#     print("Label not found in the text.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'tokens': 'TheCartPress', 'label': 1}



"['<s> Develop a token classification model to accurately identify and classify named entities label:\\n### Token:\\nTheCartPress\\n### label:\\n1</s>']"

In [45]:
import re
inputs = tokenizer(
    [
        prompt.format(
        dataset['test']['tokens'][700], # insert comment
        "", # output - blank (instead of label) for generation
        )
    ],
    return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
decoded_outputs = tokenizer.batch_decode(outputs)
decoded_outputs = str(decoded_outputs)
# Using regular expression to find the label value starting with '###'
label_match = re.search(r'### label:\n(\d+)', decoded_outputs)

if label_match:
    label_value = label_match.group(1)
    print("Label value:", label_value)
else:
    print("Label not found in the text.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Label not found in the text.


#### Many samples

In [ ]:
import re

def extract_label(text):
    label_match = re.search(r"label:\\\n(\d+)", text)
    if label_match:
        return label_match.group(1)
    else:
        return "Label not found."

for i, sample in enumerate(dataset['test']):
    inputs = tokenizer(
        [
            prompt.format(
            sample['tokens'], # insert comment
            "", # output - blank (instead of label) for generation
            )
        ],
        return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
    decoded_outputs = tokenizer.batch_decode(outputs)

    # extract "comment label" in response
    matches = re.findall(r'Token label:\n(\d)', decoded_outputs[0])
    predicted_label = int(matches[-1]) if matches else decoded_outputs

    print(f"Token {i}: {sample['tokens']}\nTrue label: {sample['label']}\nPredicted label:\n{predicted_label}\n")

In [ ]:
import re

def extract_label(text):
    label_match = re.search(r"label:\\\n(\d+)", text)
    if label_match:
        return label_match.group(1)
    else:
        return "Label not found."

predicted_labels = []  # Initialize an empty list to store predicted labels

for i, sample in enumerate(dataset['test']):
    inputs = tokenizer(
        [
            prompt.format(
            sample['tokens'], # insert comment
            "", # output - blank (instead of label) for generation
            )
        ],
        return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
    decoded_outputs = tokenizer.batch_decode(outputs)

    # extract "comment label" in response
    matches = re.findall(r'Token label:\n(\d)', decoded_outputs[0])
    predicted_label = int(matches[-1]) if matches else decoded_outputs
    predicted_labels.append(predicted_label)  # Append the predicted label to the list

    print(f"Token {i}: {sample['tokens']}\nTrue label: {sample['label']}\nPredicted label:\n{predicted_label}\n")

# Now you have all the predicted labels in the predicted_labels list
print("All predicted labels:", predicted_labels)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Token 0: TheCartPress
True label: 1
Predicted label:
['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\nTheCartPress\n### label:\n1</s>']



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Token 1: vba_extract.c
True label: 4
Predicted label:
['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\nvba_extract.c\n### label:\n4</s>']



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Token 617: 8.1x
True label: 14
Predicted label:
['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\n8.1x\n### label:\n14</s>']



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Token 618: apps/media/lib_scanner.php
True label: 4
Predicted label:
['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\napps/media/lib_scanner.php\n### label:\n4</s>']



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Token 619: MailServer
True label: 23
Predicted label:
['<s> Develop a token classification model to accurately identify and classify named entities label:\n### Token:\nMailServer\n### label:\n1</s>']



In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Actual labels
actual = dataset['test']['label'].tolist()
# Predicted labels
predicted = predicted_labels

# Calculate accuracy
accuracy = accuracy_score(actual, predicted)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(actual, predicted)
print("Precision:", precision)

# Calculate recall
recall = recall_score(actual, predicted)
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(actual, predicted)
print("F1 Score:", f1)
